In [1]:
import time
import pygame
import pygame.midi
import math
import numpy as np
from json import load

import tensorflow as tf
from keras.models import load_model

from music21 import *
from midi_util import produce_note_strings

################################################################################################################
duration_seq = [] 

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Init Plugin
Init Graph Optimizer
Init Kernel


In [ ]:
def print_midi_info():
    pygame.midi.init()
    for i in range(pygame.midi.get_count()):
        print(pygame.midi.get_device_info(i))
    
    print("Default in: ", pygame.midi.get_default_input_id())
    print("Default out: ", pygame.midi.get_default_output_id())
    pygame.midi.quit()

print_midi_info()

In [ ]:
notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

def num_to_note(num):
    note = notes[num % 12]
    note = note + str(math.ceil(num/12)-1)
    return note


In [ ]:
def take_input(in_id = None, out_id = None):
    pygame.midi.init()

    if not in_id:
        in_id = pygame.midi.get_default_input_id()

    if not out_id:
        out_id = pygame.midi.get_default_output_id()

    i = pygame.midi.Input(in_id)
    o = pygame.midi.Output(out_id,latency=0,buffer_size=1)
    o.set_instrument(0)

    pygame.display.set_mode((1,1))
    note_count = 0

    #Turn these arrays into nupy.array()s like this: final_list = np.array(list(zip(note_seq,dur_seq)))
                # Will need to map ints from keyboard to chars representing the note (ie 'C3' 'A-4' etc) 
                # then return it and in produce_note_model create a new function and map the chars to their 
                # respective ints before sending them to produce_song 

                # have to caulculate the differnece in the start and end of notes, round the result to the 
                # nearest musical "pause" then send the word along with the string for note (bpm 120)
            #Chords?
                # PLeASe nO
        #I'll need to store the notes that have been pressed to check timestamp for release of the note


        #A mid C is 60, so using mod 12 to get notes and octave is important.

    #Format: [[pitch, start, stop], [pitch,start,stop], ...]
    note_seq = [] 
    while note_count < 8:
        if i.poll(): #if a note has been presseed
            midi_event = i.read(1) #grab note played

            if midi_event[0][0][0] == 144: #if note on
                print(midi_event)
                print(num_to_note(midi_event[0][0][1]))
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1] # in milliseconds I think

                note_seq.append([note, time_stamp, 0])

                o.note_on(note, 120)               

            if midi_event[0][0][0] == 128: #if note off
                print(midi_event)
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1]
                for n in range(len(note_seq)-1, -1, -1):
                    if(note_seq[n][0] == note and note_seq[n][2] == 0):
                        note_seq[n][2] = time_stamp
                o.note_off(note, 120)
               
                note_count += 1  #DOES NOT ACCOUNT FOR CHORDS

    ################################################################################################################
    for j in note_seq:
        duration_seq.append(j[2]-j[1])

    print(note_seq)
    print(duration_seq)
    time.sleep(1) #Give last note time to play
    i.close()
    o.close()
    pygame.midi.quit()
    pygame.quit()

    return note_seq

In [2]:
json_file = open("int_to_note.json")

int_to_note = load(json_file)
note_to_int = {v: k for k, v in int_to_note.items()}

In [ ]:
note_seq = take_input(out_id=4)

In [ ]:
user_notes_int = []
user_notes_str = []
for note in note_seq:
    pitch = note[0]
    note_str = num_to_note(pitch)

    if note_str not in note_to_int:
        note_str = note_str[:-1] + "3" # B5 -> B3

    user_notes_int.append(int(note_to_int[note_str]))
    user_notes_str.append(note_str)

print(user_notes_int)
print(user_notes_str)

In [3]:
### Mocking
# user_notes_int=[33,4,33,4,33,27,33,33]

In [5]:
predicted_notes_str = produce_note_strings(user_notes_int, int_to_note, n_notes=8)
print("Complete sequence:", predicted_notes_str)
print("Predicted notes:", predicted_notes_str[len(user_notes_int):])

Complete sequence: ['F4', 'A4', 'F4', 'A4', 'F4', 'E4', 'F4', 'F4', 'D3', 'E4', 'E3', 'E-3', 'E-3', 'E-3', 'D5', 'D5']
Predicted notes: ['D3', 'E4', 'E3', 'E-3', 'E-3', 'E-3', 'D5', 'D5']


2022-02-23 14:20:33.810079: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-23 14:20:33.838219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
output_notes = []
offset = 0
for c, j in enumerate(duration_seq):
    new_note = note.Note(int_to_note[c]) 
    new_note.offset += 1
    new_note.duration = j
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

print(output_notes)